In [3]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pandas as pd
import string

class SynonymTextEditor:
    def __init__(self, root):
        """Initialize the Synonym Text Editor with GUI elements and load data."""
        self.root = root
        self.root.title("Synonym Text Editor")
        
        # Create the main text area where the user can input text
        self.text_area = tk.Text(root, wrap="word", font=("Arial", 16))
        self.text_area.pack(expand=1, fill="both")

        # Create a dropdown menu for displaying synonyms
        self.synonym_box = ttk.Combobox(root, state="readonly", font=("Arial", 12))
        self.synonym_box.pack(fill="x", padx=5, pady=5)
        self.synonym_box.bind("<<ComboboxSelected>>", self.enable_apply_button)
        self.synonym_box.set("No synonyms found yet")

        # Button to apply the selected synonym
        self.apply_button = tk.Button(root, text="Apply", command=self.apply_synonym, state="disabled")
        self.apply_button.pack(pady=5)
        
        # Load Excel file containing synonyms
        self.data = None
        self.current_word_range = None  # Stores the range of the currently selected word/phrase
        self.default_excel_path = "./excel/synonyms.xlsx"  # Default path for synonym database
        self.load_default_excel()

        # Bind events for text selection and highlighting
        self.text_area.bind("<Double-1>", self.check_selected_phrase)
        self.text_area.bind("<KeyRelease>", self.highlight_all_matches)
    
    def load_default_excel(self):
        """Load the default Excel file containing keyword synonyms."""
        try:
            self.data = pd.read_excel(self.default_excel_path)
            if "Keywords" in self.data.columns and "Category" in self.data.columns:
                # Convert keywords to lowercase for case-insensitive matching
                self.data["Keywords"] = self.data["Keywords"].astype(str).str.lower()
                print("Excel file loaded successfully.")
            else:
                messagebox.showerror("Error", "Excel file must contain 'Keywords' and 'Category' columns.")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load Excel file: {e}")
    
    def check_selected_phrase(self, event):
        """Detect if the clicked word is part of a multi-word phrase from the Excel file."""
        if self.data is None:
            return
        
        try:
            index = self.text_area.index(tk.CURRENT)  # Get cursor position in the text area
            content = self.text_area.get("1.0", "end-1c").lower()  # Retrieve full text content
            words = content.split()
            
            cursor_pos = int(index.split(".")[1])  # Extract cursor position from index
            phrase_start = phrase_end = cursor_pos
            
            # Iterate through all phrases in the dataset to find a match
            for phrase in self.data['Keywords'].dropna().values:
                if phrase in content:
                    start_idx = content.find(phrase)
                    end_idx = start_idx + len(phrase)
                    if start_idx <= cursor_pos <= end_idx:
                        phrase_start = start_idx
                        phrase_end = end_idx
                        break
            
            # Convert positions to tkinter text indices
            start_idx = f"1.0 + {phrase_start}c"
            end_idx = f"1.0 + {phrase_end}c"
            self.current_word_range = (start_idx, end_idx)
            
            # Highlight selected phrase and display synonyms
            self.highlight_selected_word(start_idx, end_idx)
            self.match_and_display_synonyms(content[phrase_start:phrase_end])
        except Exception as e:
            messagebox.showerror("Error", f"Failed to process the selection: {e}")
    
    def highlight_all_matches(self, event=None):
        """Highlight all words and phrases found in the Excel file."""
        if self.data is None:
            return
        
        content = self.text_area.get("1.0", "end-1c").lower()
        self.text_area.tag_remove("match", "1.0", "end")
        
        # Iterate through the dataset and highlight matching words/phrases
        for phrase in self.data['Keywords'].dropna().values:
            start_idx = "1.0"
            while True:
                start_idx = self.text_area.search(phrase, start_idx, stopindex="end", nocase=True)
                if not start_idx:
                    break
                end_idx = f"{start_idx} + {len(phrase)}c"
                self.text_area.tag_add("match", start_idx, end_idx)
                start_idx = end_idx
        
        self.text_area.tag_config("match", foreground="green")  # Highlight matches in green
    
    def match_and_display_synonyms(self, phrase):
        """Find and display synonyms for the selected phrase."""
        match = self.data[self.data['Keywords'].str.lower() == phrase.lower()]
        if not match.empty:
            category = match.iloc[0]['Category']  # Identify category of the selected phrase
            synonyms = self.data[self.data['Category'] == category]['Keywords'].tolist()
            self.display_synonyms(synonyms, phrase)
        else:
            self.reset_synonym_box()
    
    def display_synonyms(self, synonyms, current_word):
        """Update dropdown with synonyms, excluding the currently selected word."""
        self.synonym_box["values"] = [kw for kw in synonyms if kw.lower() != current_word.lower()]
        self.synonym_box.set("Select a synonym")
        self.apply_button.config(state="normal")
    
    def reset_synonym_box(self):
        """Clear the synonym dropdown and disable the apply button."""
        self.synonym_box["values"] = []
        self.synonym_box.set("No synonyms found yet")
        self.apply_button.config(state="disabled")
    
    def enable_apply_button(self, event):
        """Enable apply button when a synonym is selected from the dropdown."""
        if self.synonym_box.get().strip():
            self.apply_button.config(state="normal")
    
    def apply_synonym(self):
        """Replace the selected phrase with the chosen synonym."""
        selected_synonym = self.synonym_box.get().strip()
        
        if selected_synonym and self.current_word_range:
            start_idx, end_idx = self.current_word_range
            
            # Replace the text in the selected range with the chosen synonym
            self.text_area.delete(start_idx, end_idx)
            self.text_area.insert(start_idx, selected_synonym)
            
            self.reset_synonym_box()
            self.text_area.tag_remove("selected", "1.0", "end")
            self.current_word_range = None
            self.text_area.focus_set()
    
    def highlight_selected_word(self, start, end):
        """Highlight the selected word or phrase."""
        self.text_area.tag_remove("selected", "1.0", "end")
        self.text_area.tag_add("selected", start, end)
        self.text_area.tag_config("selected", background="green", foreground="white")

if __name__ == "__main__":
    root = tk.Tk()
    editor = SynonymTextEditor(root)
    root.mainloop()


Excel file loaded successfully.
